In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/val.txt
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001464.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001970.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001454.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001431.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001888.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001882.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001523.xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/img (76).xml
/kaggle/input/pedestrian-detection-data-set/Val-20200226T103730Z-001/Val/Annotations/2008_001737.xml
/kaggle/input

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saravananchandran/pedestrian-detection-data-set")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/pedestrian-detection-data-set


In [4]:
import os

# List folders/files inside the dataset directory
print("Folder content:")
for item in os.listdir(path):
    print(item)

Folder content:
Val-20200226T103730Z-001
Train-20200226T103300Z-001
Test-20200226T103653Z-001


In [5]:
for folder in ['Train-20200226T103300Z-001', 'Val-20200226T103730Z-001', 'Test-20200226T103653Z-001']:
    print(f"\n📁 {folder}")
    full_path = os.path.join(path, folder)
    for subitem in os.listdir(full_path):
        print("   └──", subitem)



📁 Train-20200226T103300Z-001
   └── Train

📁 Val-20200226T103730Z-001
   └── Val

📁 Test-20200226T103653Z-001
   └── Test


In [6]:
for parent_folder in ['Train-20200226T103300Z-001', 'Val-20200226T103730Z-001', 'Test-20200226T103653Z-001']:
    inner_path = os.path.join(path, parent_folder)
    subfolders = os.listdir(inner_path)
    print(f"\n📁 {parent_folder} contains:")
    for sub in subfolders:
        real_path = os.path.join(inner_path, sub)
        print(f"   └── {sub}")
        if os.path.isdir(real_path):
            for inner in os.listdir(real_path):
                print(f"       └── {inner}")



📁 Train-20200226T103300Z-001 contains:
   └── Train
       └── Annotations
       └── train.txt
       └── JPEGImages

📁 Val-20200226T103730Z-001 contains:
   └── Val
       └── val.txt
       └── Annotations
       └── JPEGImages

📁 Test-20200226T103653Z-001 contains:
   └── Test
       └── test.txt
       └── Annotations
       └── JPEGImages


In [7]:
import xml.etree.ElementTree as ET

sample_xml_path = os.path.join(
    path, 
    'Train-20200226T103300Z-001', 
    'Train', 
    'Annotations', 
    os.listdir(os.path.join(path, 'Train-20200226T103300Z-001', 'Train', 'Annotations'))[0]
)

tree = ET.parse(sample_xml_path)
root = tree.getroot()

# Print all objects in this annotation
print(f"\n📝 Parsed: {sample_xml_path}")
for obj in root.findall('object'):
    label = obj.find('name').text
    print("→ Label:", label)



📝 Parsed: /kaggle/input/pedestrian-detection-data-set/Train-20200226T103300Z-001/Train/Annotations/2008_000870.xml
→ Label: person


In [8]:
import os
import xml.etree.ElementTree as ET
from PIL import Image

# ✅ Root dataset folder
base_path = '/kaggle/input/pedestrian-detection-data-set'

# ✅ Mapping of split names to actual paths
split_info = {
    'train': os.path.join(base_path, 'Train-20200226T103300Z-001', 'Train'),
    'val': os.path.join(base_path, 'Val-20200226T103730Z-001', 'Val'),
    'test': os.path.join(base_path, 'Test-20200226T103653Z-001', 'Test'),
}

# ✅ Output folder for processed images
output_dir = './data'
image_size = (128, 128)

def parse_annotation(xml_path):
    """Return 'pedestrian' if any object is labeled 'person', else 'background'."""
    tree = ET.parse(xml_path)
    root = tree.getroot()
    for obj in root.findall('object'):
        label = obj.find('name').text.lower()
        if label == 'person' or label == 'pedestrian':
            return 'pedestrian'
    return 'background'

def process_split(split_name, split_path):
    print(f'\n🔄 Processing {split_name}...')

    txt_file = os.path.join(split_path, f'{split_name}.txt')
    images_dir = os.path.join(split_path, 'JPEGImages')
    annotations_dir = os.path.join(split_path, 'Annotations')

    # Make output folders
    for cls in ['pedestrian', 'background']:
        os.makedirs(os.path.join(output_dir, split_name, cls), exist_ok=True)

    # Read list of image names
    with open(txt_file, 'r') as f:
        image_names = [line.strip() for line in f]

    for name in image_names:
        img_path = os.path.join(images_dir, f'{name}.jpg')
        xml_path = os.path.join(annotations_dir, f'{name}.xml')

        if not os.path.exists(img_path) or not os.path.exists(xml_path):
            print(f'⚠️ Skipping {name} — missing file.')
            continue

        label = parse_annotation(xml_path)

        img = Image.open(img_path).convert('RGB')
        img = img.resize(image_size)

        save_path = os.path.join(output_dir, split_name, label, f'{name}.jpg')
        img.save(save_path)

    print(f'✅ Done with {split_name}.')

def main():
    for split_name, split_path in split_info.items():
        process_split(split_name, split_path)
    print('\n🎉 All images processed and saved to ./data')

if __name__ == '__main__':
    main()



🔄 Processing train...
✅ Done with train.

🔄 Processing val...
✅ Done with val.

🔄 Processing test...
✅ Done with test.

🎉 All images processed and saved to ./data


In [9]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# 🧠 Define basic transforms
transform = transforms.Compose([
    transforms.ToTensor(),  # Converts to [0,1] and channels-first [C,H,W]
])

# 📂 Directories
data_dir = './data'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

# 📦 Load datasets
train_dataset = datasets.ImageFolder(train_dir, transform=transform)
val_dataset = datasets.ImageFolder(val_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_dir, transform=transform)

# 📥 Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 🧾 Check class-to-index mapping
print("Class mapping:", train_dataset.class_to_idx)

Class mapping: {'background': 0, 'pedestrian': 1}


In [10]:
import os

print("✅ Contents of current directory:")
print(os.listdir('.'))

print("\n📂 Contents of './data':")
if os.path.exists('./data'):
    print(os.listdir('./data'))
else:
    print("❌ './data' folder not found!")

✅ Contents of current directory:
['data', '.virtual_documents']

📂 Contents of './data':
['test', 'train', 'val']


In [11]:
data_dir = './data'

In [12]:
data_dir = '/kaggle/working/data'

In [13]:
import os

for root, dirs, files in os.walk("/kaggle/working"):
    for d in dirs:
        if d in ["pedestrian", "background"]:
            print("✅ Found class folder in:", os.path.join(root, d))

✅ Found class folder in: /kaggle/working/data/test/background
✅ Found class folder in: /kaggle/working/data/test/pedestrian
✅ Found class folder in: /kaggle/working/data/train/background
✅ Found class folder in: /kaggle/working/data/train/pedestrian
✅ Found class folder in: /kaggle/working/data/val/background
✅ Found class folder in: /kaggle/working/data/val/pedestrian


In [14]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# ✅ Correct data directory
data_dir = '/kaggle/working/data'

# 🧠 Define transforms
transform = transforms.Compose([
    transforms.ToTensor()
])

# 📦 Load datasets
train_dataset = datasets.ImageFolder(os.path.join(data_dir, 'train'), transform=transform)
val_dataset   = datasets.ImageFolder(os.path.join(data_dir, 'val'), transform=transform)
test_dataset  = datasets.ImageFolder(os.path.join(data_dir, 'test'), transform=transform)

# 📥 DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# 🧾 Confirm classes
print("✅ Class mapping:", train_dataset.class_to_idx)


✅ Class mapping: {'background': 0, 'pedestrian': 1}


In [15]:
train_model(model, train_loader, val_loader, criterion, optimizer, device, num_epochs=10)

NameError: name 'train_model' is not defined

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

def evaluate_on_test(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            labels = labels.to(device)
            
            outputs = model(images)
            preds = (outputs >= 0.5).float().cpu().numpy()
            all_preds.extend(preds.flatten())
            all_labels.extend(labels.cpu().numpy())

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    
    print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=['background', 'pedestrian']))

    plt.figure(figsize=(6,5))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['background', 'pedestrian'], yticklabels=['background', 'pedestrian'])
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.title('Confusion Matrix on Test Set')
    plt.show()

# Ejecutar evaluación
evaluate_on_test(model, test_loader, device)


In [ ]:
import torch
from torch.utils.data import WeightedRandomSampler

def make_weights_for_balanced_classes(dataset):
    # Count how many samples per class
    class_counts = [0] * len(dataset.classes)
    for _, label in dataset.samples:
        class_counts[label] += 1
    
    # Calculate weight for each class: inverse frequency
    class_weights = [0.] * len(dataset.classes)
    for i in range(len(dataset.classes)):
        class_weights[i] = 1.0 / class_counts[i]
    
    # Assign a weight to each sample based on its class
    sample_weights = [0] * len(dataset.samples)
    for idx, (path, label) in enumerate(dataset.samples):
        sample_weights[idx] = class_weights[label]
    
    return sample_weights

# Create weights for train dataset
train_weights = make_weights_for_balanced_classes(train_dataset)

# Create sampler
train_sampler = WeightedRandomSampler(train_weights, num_samples=len(train_weights), replacement=True)

# Create DataLoader with sampler
train_loader_balanced = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)

In [ ]:
from torch.utils.data import WeightedRandomSampler

def make_weights_for_balanced_classes(dataset):
    class_counts = [0] * len(dataset.classes)
    for _, label in dataset.samples:
        class_counts[label] += 1

    class_weights = [0.] * len(dataset.classes)
    for i in range(len(dataset.classes)):
        class_weights[i] = 1.0 / class_counts[i]

    sample_weights = [0] * len(dataset.samples)
    for idx, (path, label) in enumerate(dataset.samples):
        sample_weights[idx] = class_weights[label]

    return sample_weights

train_weights = make_weights_for_balanced_classes(train_dataset)
train_sampler = WeightedRandomSampler(train_weights, num_samples=len(train_weights), replacement=True)

train_loader_balanced = DataLoader(train_dataset, batch_size=32, sampler=train_sampler)

In [ ]:
train_model(model, train_loader_balanced, val_loader, criterion, optimizer, device, num_epochs=10)


In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance de voltear horizontalmente
    transforms.RandomRotation(10),           # Rotación aleatoria ±10 grados
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Cambios de color
    transforms.ToTensor(),                    # Convertir a tensor y normalizar a [0,1]
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
])


In [ ]:
from torchvision import transforms

train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),  # 50% chance de voltear horizontalmente
    transforms.RandomRotation(10),           # Rotación aleatoria ±10 grados
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),  # Cambios de color
    transforms.ToTensor(),                    # Convertir a tensor y normalizar a [0,1]
])

val_test_transform = transforms.Compose([
    transforms.ToTensor(),
])


In [ ]:
train_dataset.transform = train_transform
val_dataset.transform = val_test_transform
test_dataset.transform = val_test_transform


In [ ]:
train_model(model, train_loader_balanced, val_loader, criterion, optimizer, device, num_epochs=10)


In [ ]:
evaluate_on_test(model, test_loader, device)

In [ ]:
# Save the model weights
torch.save(model.state_dict(), "pedestrian_cnn_model.pth")
print("✅ Model saved as pedestrian_cnn_model.pth")

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.net = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1),  # input: [3, 128, 128]
            nn.ReLU(),
            nn.MaxPool2d(2),                # → [16, 64, 64]
            
            nn.Conv2d(16, 32, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                # → [32, 32, 32]

            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),                # → [64, 16, 16]

            nn.Flatten(),
            nn.Linear(64 * 16 * 16, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)    # [3, 128, 128] → [16, 128, 128]
        self.pool = nn.MaxPool2d(2)                    # [16, 64, 64]
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)   # [32, 64, 64]
        # No conv3!

        self.fc1 = nn.Linear(32 * 32 * 32, 128)        # Because after 2x pooling: [32, 32, 32]
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.pool(nn.ReLU()(self.conv1(x)))  # → [16, 64, 64]
        x = self.pool(nn.ReLU()(self.conv2(x)))  # → [32, 32, 32]
        x = x.view(-1, 32 * 32 * 32)
        x = nn.ReLU()(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return self.sigmoid(x)


In [ ]:
model = CNN().to(device)
model.load_state_dict(torch.load("pedestrian_cnn_model.pth"))
model.eval()
print("✅ Model loaded successfully!")

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        inputs = inputs.view(-1)
        targets = targets.view(-1).float()

        BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        pt = torch.where(targets == 1, inputs, 1 - inputs)

        focal_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:
criterion = FocalLoss(alpha=0.25, gamma=2.0)

In [ ]:
num_epochs = 10  # define cuántas épocas entrenarás

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * inputs.size(0)
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")

In [ ]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:  # o test_loader, según quieras evaluar
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        preds = (outputs > 0.5).int()  # umbral 0.5 para binario
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convertir listas a arrays
all_preds = [int(p[0]) for p in all_preds]
all_labels = [int(l[0]) for l in all_labels]

# Matriz de confusión
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

# Reporte de clasificación
cr = classification_report(all_labels, all_preds, target_names=['background', 'pedestrian'])
print("Classification Report:\n", cr)

# Visualización
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['background', 'pedestrian'], yticklabels=['background', 'pedestrian'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
alphas = [0.1, 0.25, 0.5]
gammas = [1, 2, 5]

best_val_acc = 0
best_params = {'alpha': None, 'gamma': None}

for alpha in alphas:
    for gamma in gammas:
        print(f"\nTraining with alpha={alpha}, gamma={gamma}")
        
        # Crear modelo y optimizador fresco
        model = CNN().to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
        criterion = FocalLoss(alpha=alpha, gamma=gamma)

        # Entrenamos (puedes ajustar epochs para hacerlo más rápido)
        num_epochs = 5
        for epoch in range(num_epochs):
            model.train()
            running_loss = 0.0
            correct = 0
            total = 0
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                preds = (outputs > 0.5).float()
                correct += (preds == labels).sum().item()
                total += labels.size(0)
            epoch_acc = correct / total
            print(f"Epoch {epoch+1}/{num_epochs} - Train Acc: {epoch_acc:.4f}")

        # Evaluar en validación
        model.eval()
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(inputs)
                preds = (outputs > 0.5).float()
                correct_val += (preds == labels).sum().item()
                total_val += labels.size(0)
        val_acc = correct_val / total_val
        print(f"Validation Accuracy: {val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_params['alpha'] = alpha
            best_params['gamma'] = gamma

print(f"\nBest Validation Accuracy: {best_val_acc:.4f} with alpha={best_params['alpha']} and gamma={best_params['gamma']}")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Asumiendo que CNN está definido, device configurado, y train_loader/val_loader listos

# Definición del Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=1, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
        self.bce = nn.BCELoss(reduction='none')
    def forward(self, inputs, targets):
        BCE_loss = self.bce(inputs, targets)
        pt = torch.where(targets == 1, inputs, 1 - inputs)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# Inicializa modelo, criterio, optimizador
model = CNN().to(device)
criterion = FocalLoss(alpha=0.25, gamma=1)
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# Entrenamiento
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total

    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {epoch_loss:.4f} | Train Acc: {epoch_acc:.4f}")

# Guardar modelo
torch.save(model.state_dict(), "pedestrian_cnn_model_focal.pth")
print("✅ Modelo guardado como 'pedestrian_cnn_model_focal.pth'")

# Evaluación en validación
model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        preds = (outputs > 0.5).int()
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Convertir listas a formato esperado para sklearn
all_preds = [int(p[0]) for p in all_preds]
all_labels = [int(l[0]) for l in all_labels]

# Matriz de confusión y reporte
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)

cr = classification_report(all_labels, all_preds, target_names=['background', 'pedestrian'])
print("Classification Report:\n", cr)

# Visualización
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['background', 'pedestrian'],
            yticklabels=['background', 'pedestrian'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support

model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        all_probs.extend(outputs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_probs = np.array([p[0] for p in all_probs])
all_labels = np.array([l[0] for l in all_labels])

thresholds = np.linspace(0, 1, 50)
results = []

for thresh in thresholds:
    preds = (all_probs >= thresh).astype(int)
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, preds, average='binary')
    results.append((thresh, precision, recall, f1))

# Mostrar los mejores umbrales según F1-score
results = sorted(results, key=lambda x: x[3], reverse=True)
print("Top 5 thresholds by F1-score:")
for thresh, precision, recall, f1 in results[:5]:
    print(f"Threshold: {thresh:.2f} | Precision: {precision:.3f} | Recall: {recall:.3f} | F1-score: {f1:.3f}")

# Opcional: graficar Precision, Recall y F1 vs umbral
import matplotlib.pyplot as plt

ths, precisions, recalls, f1s = zip(*results)
plt.plot(ths, precisions, label='Precision')
plt.plot(ths, recalls, label='Recall')
plt.plot(ths, f1s, label='F1-score')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.legend()
plt.title('Precision, Recall, F1 vs Threshold')
plt.show()

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import seaborn as sns

# Define el umbral óptimo encontrado
best_threshold = 0.45  # Cambia a 0.53 o el que prefieras

model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        all_probs.extend(outputs.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

all_probs = np.array([p[0] for p in all_probs])
all_labels = np.array([l[0] for l in all_labels])

# Aplica el umbral para obtener predicciones binarias
preds = (all_probs >= best_threshold).astype(int)

# Calcula matriz de confusión y reporte
cm = confusion_matrix(all_labels, preds)
report = classification_report(all_labels, preds, target_names=['background', 'pedestrian'])

print("Confusion Matrix:")
print(cm)
print("\nClassification Report:")
print(report)

# Visualiza la matriz de confusión
plt.figure(figsize=(6,5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['background', 'pedestrian'], yticklabels=['background', 'pedestrian'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title(f'Confusion Matrix (threshold={best_threshold})')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming you have:
# model (trained with focal loss, alpha=0.25, gamma=1)
# test_loader (DataLoader for your test or val set)
# device (cpu or cuda)

model.eval()
all_probs = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        probs = outputs.squeeze().cpu().numpy()  # Assuming output is a sigmoid scalar per sample
        all_probs.extend(probs)
        all_labels.extend(labels.cpu().numpy())

all_probs = np.array(all_probs)
all_labels = np.array(all_labels)

thresholds = np.linspace(0, 1, 101)
precisions, recalls, f1s = [], [], []

for thresh in thresholds:
    preds = (all_probs >= thresh).astype(int)
    precisions.append(precision_score(all_labels, preds))
    recalls.append(recall_score(all_labels, preds))
    f1s.append(f1_score(all_labels, preds))

plt.figure(figsize=(10,6))
plt.plot(thresholds, precisions, label='Precision', color='blue')
plt.plot(thresholds, recalls, label='Recall', color='green')
plt.plot(thresholds, f1s, label='F1-score', color='red')
plt.xlabel('Threshold')
plt.ylabel('Score')
plt.title('Precision, Recall, F1-score vs Threshold (Focal Loss α=0.25, γ=1)')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# 🧪 Evaluate the model on test set
model.eval()
y_true = []
y_pred = []
y_probs = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        probs = outputs.squeeze().cpu().numpy()
        preds = (probs >= 0.5).astype(int)

        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds)
        y_probs.extend(probs)

# 📊 Confusion Matrix
cm = confusion_matrix(y_true, y_pred)
print("Confusion Matrix:")
print(cm)

# 📄 Classification Report
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=['background', 'pedestrian']))


In [ ]:
OK LETS TRAIN AGAIN EVERYTHING NOW WITH THE COMPARISSONS AND RESULTS

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# ✅ 1. Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ 2. Define CNN
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # (128x128) → (64x64)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)
        self.fc1 = nn.Linear(32 * 32 * 32, 128)
        self.fc2 = nn.Linear(128, 1)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # → 64x64
        x = self.pool(F.relu(self.conv2(x)))  # → 32x32
        x = x.view(-1, 32 * 32 * 32)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# ✅ 3. Focal Loss
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.25, gamma=1):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        eps = 1e-7
        inputs = inputs.clamp(min=eps, max=1. - eps)
        targets = targets.float()
        bce = - (targets * torch.log(inputs) + (1 - targets) * torch.log(1 - inputs))
        focal = self.alpha * (1 - inputs) ** self.gamma * bce
        return focal.mean()

# ✅ 4. Transforms and Datasets
transform = transforms.Compose([
    transforms.ToTensor()
])

data_dir = '/kaggle/working/data'  # adjust if needed
train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
val_dataset   = datasets.ImageFolder(root=f'{data_dir}/val', transform=transform)
test_dataset  = datasets.ImageFolder(root=f'{data_dir}/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

# ✅ 5. Initialize
model = CNN().to(device)
criterion = FocalLoss(alpha=0.25, gamma=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# ✅ 6. Training loop
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss, correct, total = 0.0, 0, 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        preds = (outputs > 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    acc = correct / total
    print(f"Epoch {epoch+1}/{num_epochs} | Loss: {running_loss:.4f} | Train Acc: {acc:.4f}")

# ✅ 7. Save model
torch.save(model.state_dict(), "pedestrian_cnn_model_focal_best.pth")
print("✅ Model saved!")

# ✅ 8. Evaluate on test set
model.eval()
all_preds, all_labels = [], []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        preds = (outputs > 0.5).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.numpy())

# ✅ 9. Confusion matrix and report
cm = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=['background', 'pedestrian'])
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

# Optional: plot confusion matrix
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['background','pedestrian'], yticklabels=['background','pedestrian'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
criterion = FocalLoss(alpha=0.15, gamma=1)  # Instead of 0.25


In [ ]:
inputs = inputs.clamp(min=1e-5, max=1 - 1e-5)  # To avoid log(0)


In [ ]:
model = CNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
# Adjust Focal Loss to be more stable
class FocalLoss(nn.Module):
    def __init__(self, alpha=0.15, gamma=1):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma

    def forward(self, inputs, targets):
        eps = 1e-5
        inputs = inputs.clamp(min=eps, max=1. - eps)
        targets = targets.float()
        bce = - (targets * torch.log(inputs) + (1 - targets) * torch.log(1 - inputs))
        focal = self.alpha * (1 - inputs) ** self.gamma * bce
        return focal.mean()

# Then retrain the model from scratch
model = CNN().to(device)
criterion = FocalLoss(alpha=0.15, gamma=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [ ]:
cm = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=['background', 'pedestrian'])
print("Confusion Matrix:\n", cm)
print("Classification Report:\n", report)

# Optional: plot confusion matrix
plt.figure(figsize=(6,4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['background','pedestrian'], yticklabels=['background','pedestrian'])
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
import torch
import torch.nn as nn

# Make sure to define your CNN model class exactly as it was when you saved the model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)  # Adjust this if image size changed
        self.fc2 = nn.Linear(128, 1)  # Binary classification output

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)  # Flatten
        x = torch.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x.squeeze()

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Instantiate the model and load weights
model = CNN().to(device)
model.load_state_dict(torch.load('pedestrian_cnn_model.pth'))
model.eval()

# Now you can run evaluation on your validation or test set using this model
# Example evaluation snippet:

from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in val_loader:  # or test_loader
        inputs = inputs.to(device)
        labels = labels.to(device).float()
        outputs = model(inputs)
        preds = (outputs > 0.5).float()  # threshold 0.5
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
report = classification_report(all_labels, all_preds, target_names=['background', 'pedestrian'])

print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n", report)


In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
input_size = 3 * 128 * 128  # Flattened image size
hidden_size1 = 512
hidden_size2 = 128
output_size = 1  # Binary output
lr = 0.001
batch_size = 32
num_epochs = 10

# Dataset & DataLoader with flatten transform
transform = transforms.Compose([
    transforms.ToTensor(),               # Convert to tensor
    transforms.Resize((128, 128)),      # Just in case
    transforms.Lambda(lambda x: x.view(-1))  # Flatten image tensor to vector
])

# Paths to your dataset folders
data_dir = './data'
train_dataset = datasets.ImageFolder(root=f'{data_dir}/train', transform=transform)
val_dataset = datasets.ImageFolder(root=f'{data_dir}/val', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Define MLP model
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(hidden_size2, output_size),
            nn.Sigmoid()  # For binary classification
        )
    
    def forward(self, x):
        return self.net(x).squeeze()

# Instantiate model, loss, optimizer
model = MLP().to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

# Training loop
def train(model, loader):
    model.train()
    running_loss = 0
    correct = 0
    total = 0
    
    for images, labels in loader:
        images, labels = images.to(device), labels.float().to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item() * images.size(0)
        preds = (outputs >= 0.5).float()
        correct += (preds == labels).sum().item()
        total += labels.size(0)
    
    return running_loss / total, correct / total

# Validation loop
def validate(model, loader):
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.float().to(device)
            outputs = model(images)
            preds = (outputs >= 0.5).float()
            correct += (preds == labels).sum().item()
            total += labels.size(0)
    
    return correct / total

# Training process
for epoch in range(num_epochs):
    train_loss, train_acc = train(model, train_loader)
    val_acc = validate(model, val_loader)
    print(f'Epoch {epoch+1}/{num_epochs} | Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}')

# Save model
torch.save(model.state_dict(), 'pedestrian_mlp_model.pth')
print("✅ MLP Model trained and saved.")


Epoch 1/10 | Loss: 3.4083 | Train Acc: 0.5477 | Val Acc: 0.4188
Epoch 2/10 | Loss: 0.7758 | Train Acc: 0.5159 | Val Acc: 0.4875
Epoch 3/10 | Loss: 0.6891 | Train Acc: 0.5625 | Val Acc: 0.5125
Epoch 4/10 | Loss: 0.6815 | Train Acc: 0.5413 | Val Acc: 0.5312
Epoch 5/10 | Loss: 0.6780 | Train Acc: 0.5625 | Val Acc: 0.5813
Epoch 6/10 | Loss: 0.6636 | Train Acc: 0.5794 | Val Acc: 0.5625
Epoch 7/10 | Loss: 0.6521 | Train Acc: 0.5890 | Val Acc: 0.5125
Epoch 8/10 | Loss: 0.6578 | Train Acc: 0.5932 | Val Acc: 0.6000
Epoch 9/10 | Loss: 0.6553 | Train Acc: 0.6017 | Val Acc: 0.5813
Epoch 10/10 | Loss: 0.6219 | Train Acc: 0.6144 | Val Acc: 0.6375
✅ MLP Model trained and saved.


In [18]:
torch.save(model.state_dict(), "mlp_model.pth")

In [19]:
torch.save(model.state_dict(), "mlp_model.pth")
print("✅ MLP Model saved as 'mlp_model.pth'")

✅ MLP Model saved as 'mlp_model.pth'


In [20]:
# Recreate the MLP model architecture
model = MLP().to(device)

# Load saved weights
model.load_state_dict(torch.load("mlp_model.pth"))
model.eval()

print("✅ MLP model loaded and ready for inference!")

✅ MLP model loaded and ready for inference!


In [21]:
import torch
from sklearn.metrics import confusion_matrix, classification_report
import numpy as np

# Make sure model is in eval mode
model.eval()

all_preds = []
all_labels = []

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Confusion Matrix
cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:")
print(cm)

# Classification Report
report = classification_report(all_labels, all_preds, target_names=test_dataset.classes)
print("\nClassification Report:")
print(report)


Confusion Matrix:
[[  8  70]
 [ 31 126]]

Classification Report:
              precision    recall  f1-score   support

  background       0.21      0.10      0.14        78
  pedestrian       0.64      0.80      0.71       157

    accuracy                           0.57       235
   macro avg       0.42      0.45      0.43       235
weighted avg       0.50      0.57      0.52       235



In [22]:
import pandas as pd

cm_df = pd.DataFrame(cm, index=test_dataset.classes, columns=test_dataset.classes)
print("Confusion Matrix:")
print(cm_df)


Confusion Matrix:
            background  pedestrian
background           8          70
pedestrian          31         126
